### 1. Dataset creation

Since no datasets existed, the idea was to create my own using blender and check if automatically generated dataset could be created from taking pictures of assets in Blender. Therefore multiple 3D object of pokémons have been downloaded and a script called blender_script has been coded to make multiple pictures of those pokémons creating the needed dataset.
The command to create picture is `blender --background --python "blender_script.py"` (from project root). Multiple datasets have been made (from dataset1 to dataset6) with small modifications to create the one that gives best results.

As proof of concept only 9 Pokémon models have been downloaded: Abra, Chansey, Eevee, Jigglypuff, Pidgeotto, Pidgey, Psyduck, Rattata, Vulpix.
Those Pokémons corresponds to the entities that can be encountered in route 8 in the game "Pokémon: Let's go, Eevee".

### 2. Data augmentation and COCO to YOLO conversion

To ensure there was enough data and to train the model more efficiently, datasets were subjected to data augmentation using albumentations. Three augmented version of each images are attempted but it may happen that no augmentation have been triggered resulting in an omission in this case. Those initially COCO format datasets were during reformatted in YOLO datasets containing an image with a 80/20 split ratio for training and validation
sets.


In [16]:
import os
import cv2
import json
import shutil
import yaml
import random
from tqdm import tqdm
from pathlib import Path
import albumentations as A

COCO_ANNOTS = "datasets/dataset6-annotations.json"
IMAGES_DIR = "datasets/dataset6-images"
YOLO_OUT = "datasets/dataset6-neg_bg"
YOLO_OUT_AUG = "datasets/dataset6-neg_bg_aug"

NUM_AUGS_PER_IMAGE = 3
TRAIN_RATIO = 0.8
SEED = 511

augmentations = [
    A.OneOf([
        A.GaussNoise(),
        A.ISONoise()
    ]),
    A.CoarseDropout(num_holes_range=(1, 3), fill=(123, 176, 84), p=0.25),
    A.RandomBrightnessContrast(),
    A.RandomGamma(),
    A.HueSaturationValue(),
    A.Blur(),
    A.MotionBlur(),
    A.ToGray(p=0.2),
    A.Affine(shear=(-10, 10), p=0.3),
    A.Affine(scale=(0.5, 1), p=0.3),
    A.ElasticTransform(p=0.2)
]

transform = A.Compose(
    augmentations,
    bbox_params=A.BboxParams(format='coco', label_fields=['category_ids'], min_visibility=0.05)
)

def coco_to_yolo(image_list, split_name, images_input_dir,
                images_output_dir, labels_output_dir,
                image_annots, category_mapping):

    for image_info in tqdm(image_list, desc=f"Converting {split_name} set"):
        image_id = image_info['id']
        image_filename = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']

        if os.path.isdir(images_input_dir):
            source_image_path = None
            for root, _, files in os.walk(images_input_dir):
                if image_filename in files:
                    source_image_path = os.path.join(root, image_filename)
                    break
        else:
            source_image_path = images_input_dir if os.path.basename(images_input_dir) == image_filename else None

        if not source_image_path or not os.path.exists(source_image_path):
            continue

        dest_image_path = os.path.join(images_output_dir, image_filename)
        shutil.copy2(source_image_path, dest_image_path)

        label_filename = os.path.splitext(image_filename)[0] + ".txt"
        label_path = os.path.join(labels_output_dir, label_filename)

        yolo_annotations = []
        if image_id in image_annots:
            for ann in image_annots[image_id]:
                coco_bbox = ann['bbox']
                x, y, w, h = coco_bbox

                # Convert to YOLO format
                center_x = (x + w/2) / image_width
                center_y = (y + h/2) / image_height
                norm_width = w / image_width
                norm_height = h / image_height

                coco_category_id = ann['category_id']
                yolo_class_id = category_mapping[coco_category_id]

                yolo_line = f"{yolo_class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}"
                yolo_annotations.append(yolo_line)

        with open(label_path, 'w') as f:
            f.write('\n'.join(yolo_annotations))
            if yolo_annotations:
                f.write('\n')
    return

def data_aug_and_split(input_annot_file, input_images_dir, output_yolo_dir, output_yolo_aug_dir):
    random.seed(SEED)
    with open(input_annot_file, 'r') as f:
        coco_data = json.load(f)

    categories = coco_data['categories']
    category_mapping = {}
    class_names = []
    sorted_categories = sorted(categories, key=lambda x: x['id'])
    for idx, category in enumerate(sorted_categories):
        category_mapping[category['id']] = idx
        class_names.append(category['name'])

    # Split images into train and validation
    images_list = list(coco_data['images'])
    random.shuffle(images_list)
    train_count = int(len(images_list) * TRAIN_RATIO)
    train_images = images_list[:train_count]
    val_images = images_list[train_count:]

    image_id_to_annotations = {}
    for annot in coco_data['annotations']:
        image_id = annot['image_id']
        image_id_to_annotations.setdefault(image_id, []).append(annot)

    for yolo_dir in [output_yolo_dir, output_yolo_aug_dir]:
        os.makedirs(os.path.join(yolo_dir, "images", "train"), exist_ok=True)
        os.makedirs(os.path.join(yolo_dir, "images", "val"), exist_ok=True)
        os.makedirs(os.path.join(yolo_dir, "labels", "train"), exist_ok=True)
        os.makedirs(os.path.join(yolo_dir, "labels", "val"), exist_ok=True)

    # Process regular dataset
    print(f"Processing images without augmentation...")
    coco_to_yolo(train_images, "training regular", input_images_dir, 
                os.path.join(output_yolo_dir, "images", "train"),
                os.path.join(output_yolo_dir, "labels", "train"),
                image_id_to_annotations, category_mapping)

    coco_to_yolo(val_images, "validation regular", input_images_dir,
                os.path.join(output_yolo_dir, "images", "val"),
                os.path.join(output_yolo_dir, "labels", "val"),
                image_id_to_annotations, category_mapping)

    # Create dataset.yaml for regular dataset
    yaml_content = {
        'path': output_yolo_dir,
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),
        'names': class_names
    }
    with open(os.path.join(output_yolo_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    # Augment training images
    print(f"Processing images with augmentation...")
    for image in tqdm(train_images, desc="Augmenting training images"):
        if os.path.isdir(input_images_dir):
            img_path = None
            for root, _, files in os.walk(input_images_dir):
                if image['file_name'] in files:
                    img_path = os.path.join(root, image['file_name'])
                    break
        else:
            img_path = input_images_dir if os.path.basename(input_images_dir) == image['file_name'] else None

        image_id = image['id']
        annots = image_id_to_annotations.get(image_id, [])

        img = cv2.imread(img_path)
        if img is None:
            print(f"Couldn't read {img_path}")
            continue

        if annots:
            bboxes = [ann['bbox'] for ann in annots]
            category_ids = [ann['category_id'] for ann in annots]
        else:
            bboxes = []
            category_ids = []

        for i in range(NUM_AUGS_PER_IMAGE + 1):
            try:
                if i == 0:
                    aug_img = img.copy()
                    aug_bboxes = bboxes
                    aug_categories = category_ids
                else:
                    if bboxes:
                        augmented = transform(image=img, bboxes=bboxes, category_ids=category_ids)
                        aug_img = augmented['image']
                        aug_bboxes = augmented['bboxes']
                        aug_categories = augmented['category_ids']
                    else:
                        bg_transform = A.Compose(augmentations)
                        augmented = bg_transform(image=img)
                        aug_img = augmented['image']
                        aug_bboxes = []
                        aug_categories = []

                new_filename = f"{Path(image['file_name']).stem}_aug{i}.png"
                new_img_path = os.path.join(output_yolo_aug_dir, "images", "train", new_filename)
                cv2.imwrite(new_img_path, aug_img)

                label_filename = f"{Path(image['file_name']).stem}_aug{i}.txt"
                label_path = os.path.join(output_yolo_aug_dir, "labels", "train", label_filename)

                yolo_annotations = []
                for bbox, cat_id in zip(aug_bboxes, aug_categories):
                    x, y, w, h = bbox
                    center_x = (x + w/2) / aug_img.shape[1]
                    center_y = (y + h/2) / aug_img.shape[0]
                    norm_width = w / aug_img.shape[1]
                    norm_height = h / aug_img.shape[0]
                    yolo_class_id = category_mapping[cat_id]
                    yolo_line = f"{yolo_class_id} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}"
                    yolo_annotations.append(yolo_line)

                with open(label_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations))
                    if yolo_annotations:
                        f.write('\n')

            except Exception as e:
                print(f"Error while Augmenting {img_path} aug{i}: {e}")

    print("Copying validation images to augmented dataset...")
    coco_to_yolo(val_images, "validation", input_images_dir,
                os.path.join(output_yolo_aug_dir, "images", "val"),
                os.path.join(output_yolo_aug_dir, "labels", "val"),
                image_id_to_annotations, category_mapping)

    # Create dataset.yaml for augmented dataset
    yaml_content_aug = {
        'path': output_yolo_aug_dir,
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),
        'names': class_names
    }
    with open(os.path.join(output_yolo_aug_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(yaml_content_aug, f, default_flow_style=False)

# Process both datasets
if os.path.exists(COCO_ANNOTS) and os.path.exists(IMAGES_DIR):
    print("Processing dataset...")
    data_aug_and_split(COCO_ANNOTS, IMAGES_DIR, YOLO_OUT, YOLO_OUT_AUG)
    print("Dataset processing complete!")

Processing dataset without foreground...
Processing images without augmentation...


Converting validation regular set: 100%|██████████| 4587/4587 [00:51<00:00, 89.58it/s] 


Processing images with augmentation...


Augmenting training images: 100%|██████████| 18345/18345 [44:42<00:00,  6.84it/s] 


Copying validation images to augmented dataset...


Converting validation set: 100%|██████████| 4587/4587 [00:49<00:00, 91.76it/s] 

Dataset without foreground processing complete!


### 3. Check for GPU usage

Since training was too low, it has been constated that the cpu was in fact used for training. Those few lines permit to be sure that the GPU is being used for training.

In [4]:
import torch

if torch.cuda.is_available():
    print(f"GPU {torch.cuda.get_device_name(0)} will be used for training.")
else:
    print("CPU will be used for training.")

GPU NVIDIA GeForce RTX 3060 Ti will be used for training.


While training different models one after the other without restarting the computer, the computer had tendency to crash during training. Those small functions should help preventing that by checking RAM usage and by emptying cache.

In [19]:
import torch
import psutil
import gc

def check_memory():
    memory = psutil.virtual_memory()
    print(f"Total RAM: {memory.total / (1024**3):.1f} GB")
    print(f"Available RAM: {memory.available / (1024**3):.1f} GB")
    print(f"Used RAM: {memory.used / (1024**3):.1f} GB ({memory.percent}%)")


def cleanup():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

check_memory()
cleanup()

Total RAM: 15.9 GB
Available RAM: 9.1 GB
Used RAM: 6.8 GB (42.7%)


### 4. Install and train model

The model that has been chosen is yolo11n. The best performing model has also been trained on yolo11s and an attempt has been made to use yolo11m but this model was too big for the ressources I have.

In [ ]:
from ultralytics import YOLO
import torch
import os

IN_MODELS_DIR = "downloaded_models"
OUT_MODELS_DIR = "trained_models"

config = {
    "imgsz": 640,
    "epochs": 50,
    "batch": 16,
    "device": 0 if torch.cuda.is_available() else 'cpu',
    "save": True,
    "exist_ok": True,
    "verbose": True,
    "workers": 4,
    "cache": 'disk',
}

# Models to download
models_name = ["yolo11n", "yolo11s"]

# Download models
for model_name in models_name:
    model_path = f"{IN_MODELS_DIR}/{model_name}.pt"
    os.makedirs(f'{IN_MODELS_DIR}', exist_ok=True)
    model = YOLO(model_name)
    model.save(model_path)
    print(f"Downloaded {model_name} in {IN_MODELS_DIR}/")
    os.remove(f"{model_name}.pt")

# Define the training function
def train_model(model_name, dataset_name, dataset_path):
    print(f"Training {model_name}")
    results = None
    try:
        model = YOLO(f"{IN_MODELS_DIR}/{model_name}.pt")
        print(f"Loaded {model_name}.pt successfully.")
    except Exception as e:
        print(f"Failed to load {model_name}.pt: {e}")
        return

    print(f"Training on {dataset_name} dataset...")
    print(f"Dataset: {dataset_path}")

    project_name = OUT_MODELS_DIR
    run_name = f"{model_name}_{dataset_name}"
    results_summary = {}

    try:
        results = model.train(
            data=dataset_path,
            project=project_name,
            name=run_name,
            **config
        )

        if results:
            results_summary = {
                "model": model_name,
                "dataset": dataset_name,
                "project": project_name,
                "results": results,
                "status": "Completed"
            }
            print(f"{model_name} training completed on {dataset_name}")

        else:
            print(f"Training completed but no results returned")

    except Exception as e:
        error_msg = str(e)
        print(f"Training failed for {model_name} on {dataset_name}")
        print(f"Error: {error_msg}")

        results_summary = {
            "model":model_name,
            "dataset": dataset_name,
            "project": project_name,
            "results": None,
            "status": f"Failed: {error_msg}",
            "error": error_msg
        }
        return
    finally:
        summary = results_summary
        print(f"\n{model_name} using {dataset_name}:")
        print(f"   Model: {summary['model']}")
        print(f"   Dataset: {summary['dataset']}")
        print(f"   Status: {summary['status']}")
        print(f"   Project: {summary['project']}")
        print(f"   Status: {summary['status']}")
        print(f"   Error: {summary.get('error', 'No error')}")
        if hasattr(results, 'results_dict'):
            metrics = results.results_dict
            print(f"   Key Metrics:")
            for key, value in metrics.items():
                if isinstance(value, (int, float)):
                    print(f"      {key}: {value:.4f}")

        print("\nTraining completed!\n")

100%|██████████| 5.35M/5.35M [00:00<00:00, 55.3MB/s]



Downloaded yolo11n in downloaded_models/


100%|██████████| 18.4M/18.4M [00:00<00:00, 82.9MB/s]



Downloaded yolo11s in downloaded_models/


100%|██████████| 38.8M/38.8M [00:00<00:00, 70.8MB/s]



Downloaded yolo11m in downloaded_models/


##### Model training

The model yolo11n have been retrained on multiple different datasets, searching for the most suitable one

In [ ]:
check_memory()
train_model("yolo11n", "dataset6-neg_bg_aug", "datasets/dataset6-neg_bg_aug/dataset.yaml")
cleanup()

Total RAM: 15.9 GB
Available RAM: 6.4 GB
Used RAM: 9.5 GB (59.5%)
Training yolo11n
Loaded yolo11n.pt successfully.
Training on datasets/dataset6-neg_bg_aug/dataset.yaml dataset...
Dataset: datasets/dataset6-neg_bg_aug/dataset.yaml
New https://pypi.org/project/ultralytics/8.3.179 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.171  Python-3.13.1 torch-2.9.0.dev20250730+cu129 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/dataset6-neg_bg_aug/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4

100%|██████████| 5.35M/5.35M [00:00<00:00, 47.8MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.20.1 ms, read: 74.963.1 MB/s, size: 418.9 KB)


train: Scanning C:\Users\erivd\Documents\Deep Learning Project\datasets\dataset6-neg_bg_aug\labels\train... 73380 images, 797 backgrounds, 0 corrupt: 100%|██████████| 73380/73380 [02:05<00:00, 586.01it/s]


train: New cache created: C:\Users\erivd\Documents\Deep Learning Project\datasets\dataset6-neg_bg_aug\labels\train.cache


...

      50/50      3.07G     0.6908     0.4288     0.9673          4        640: 100%|██████████| 4587/4587 [09:18<00:00,  8.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 144/144 [00:25<00:00,  5.59it/s]

                   all       4587       4534      0.991      0.956      0.986      0.886



50 epochs completed in 7.869 hours.
Optimizer stripped from trained_models\yolo11n_datasets\dataset6-neg_bg_aug\dataset.yaml\weights\last.pt, 5.5MB
Optimizer stripped from trained_models\yolo11n_datasets\dataset6-neg_bg_aug\dataset.yaml\weights\best.pt, 5.5MB

Validating trained_models\yolo11n_datasets\dataset6-neg_bg_aug\dataset.yaml\weights\best.pt...
Ultralytics 8.3.171  Python-3.13.1 torch-2.9.0.dev20250730+cu129 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 144/144 [00:27<00:00,  5.15it/s]


                   all       4587       4534      0.991      0.956      0.986      0.886
                  Abra        487        487      0.994      0.973       0.99      0.902
               Chansey        487        487      0.991      0.945      0.987      0.877
                 Eevee        510        510      0.992      0.965      0.991      0.926
            Jigglypuff        478        478       0.98      0.979      0.993      0.936
             Pidgeotto        523        523      0.991      0.933      0.981      0.861
                Pidgey        532        532      0.994       0.95      0.984      0.874
               Psyduck        510        510      0.994      0.963      0.992      0.895
               Rattata        484        484      0.996      0.915      0.964      0.763
                Vulpix        523        523      0.989      0.981      0.993      0.943
Speed: 0.6ms preprocess, 1.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to trained_mod

The best performant dataset on yolo11n has been used to retrained the bigger model yolo11s to examine the difference on performance with yolo11n model.

In [ ]:
check_memory()
train_model("yolo11s", "dataset6bis-neg_bg_aug", "datasets/dataset6bis-neg_bg_aug/dataset.yaml")
cleanup()

Total RAM: 15.9 GB
Available RAM: 7.5 GB
Used RAM: 8.4 GB (52.7%)
Training yolo11s
Loaded yolo11s.pt successfully.
Training on datasets/dataset6-neg_bg_aug/dataset.yaml dataset...
Dataset: datasets/dataset6-neg_bg_aug/dataset.yaml
New https://pypi.org/project/ultralytics/8.3.179 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.171  Python-3.13.1 torch-2.9.0.dev20250730+cu129 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/dataset6-neg_bg_aug/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4

train: Scanning C:\Users\erivd\Documents\Deep Learning Project\datasets\dataset6-neg_bg_aug\labels\train.cache... 73380 images, 797 backgrounds, 0 corrupt: 100%|██████████| 73380/73380 [00:00<?, ?it/s]


WARNING train: 126.0GB disk space required, with 50% safety margin but only 53.6/895.0GB free, not caching images to disk
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.30.1 ms, read: 64.136.5 MB/s, size: 45.1 KB)


val: Scanning C:\Users\erivd\Documents\Deep Learning Project\datasets\dataset6-neg_bg_aug\labels\val.cache... 4587 images, 53 backgrounds, 0 corrupt: 100%|██████████| 4587/4587 [00:00<?, ?it/s]
val: Caching images (5.2GB Disk): 100%|██████████| 4587/4587 [00:00<00:00, 21559.60it/s]


Plotting labels to trained_models\yolo11s_datasets\dataset6-neg_bg_aug\dataset.yaml\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to trained_models\yolo11s_datasets\dataset6-neg_bg_aug\dataset.yaml
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      34/50      4.02G     0.8047     0.5692     0.9984          4        640: 100%|██████████| 4587/4587 [13:45<00:00,  5.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 144/144 [00:30<00:00,  4.68it/s]

                   all       4587       4534      0.995      0.965      0.989        0.9



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      35/50      3.99G     0.7958     0.5618     0.9937         31        640:  60%|█████▉    | 2751/4587 [08:14<05:28,  5.60it/s]

The resulting model called `dataset6bis-neg_bg_aug` has been reused on 50 manually labeled images from the game "Pokémon: Let's go, Eevee" to create a last model called `Pokédetector.pt` powerful enough to detect Pokémon efficiently in the corresponding game. This resulting model can be used to create a database for training an AI model from scratch.

### 5. `Pokédetector` usage for Dataset Creation

In [4]:
import os
from glob import glob
from ultralytics import YOLO
import cv2

SRC_DIR = "final_dataset/original_images"
OUT_PROJECT = "final_dataset"
OUT_RUN_NAME = ""

IOU_THRESHOLD = 0.4
CONF_THRESHOLD = 0.7

os.makedirs(OUT_PROJECT, exist_ok=True)

model = YOLO("trained_models/last_model/pokedetector.pt")

def get_image_dimensions(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot open image: {image_path}")
    height, width = img.shape[:2]

    return width, height

image_files = sorted(glob(os.path.join(SRC_DIR, "*.*")))
for img_path in image_files:
    w, h = get_image_dimensions(img_path)
    model.predict(
        source=img_path,
        imgsz=(w, h),
        conf=CONF_THRESHOLD,
        iou=IOU_THRESHOLD,
        project=OUT_PROJECT,
        name=OUT_RUN_NAME,
        exist_ok=True,
        save=True,
        save_txt=True,
        verbose=False
    )

WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
1 label saved to final_dataset\predict\labels
WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
2 labels saved to final_dataset\predict\labels
WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
3 labels saved to final_dataset\predict\labels
WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
4 labels saved to final_dataset\predict\labels
WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
5 labels saved to final_dataset\predict\labels
WARNING imgsz=[1920, 1080] must be multiple of max stride 32, updating to [1920, 1088]
Results saved to final_dataset\predict
6 labels s